# Tidal Potential Comparison

## Input Parameters

In [2]:
import numpy as np

from TidalPy.constants import M_earth, R_earth, G, mass_trap1
from TidalPy.toolbox.conversions import days2rads, rads2days, orbital_motion2semi_a
from TidalPy.rheology.complex_compliance import known_models as known_complex_compliances

# Orbital Properties
orbital_freq_baseline = days2rads(6.)
spin_freq_baseline = orbital_freq_baseline
eccentricity_baseline = 0.3
obliquity_baseline = 0.
host_mass = mass_trap1

# Planetary Properties
radius = R_earth
mass = M_earth
volume = (4. / 3.) * np.pi * radius**3
density_bulk = mass / volume

# Material Properties
mantle_density = 3300.
mantle_viscosity = 1.e21
mantle_shear = 50.e9
mantle_bulk_mod = 100.e9
mantle_rheology = 'maxwell'

# Numerical Properties
planet_N = 50
latitude_N = 30
longitude_N = 31
time_N = 32

# Build Static Domains
longitude_deg = np.linspace(0., 360., longitude_N)
latitude_deg = np.linspace(-89.9, 89.9, latitude_N)
longitude_rad = np.radians(longitude_deg)
latitude_rad = np.radians(latitude_deg)
colatitude_deg = 90. - latitude_deg
colatitude_rad = np.radians(colatitude_deg)

# General Parameters - y calculation
order_l = 2
use_kamata = True
use_julia = False
use_numba_integrator = False
verbose = False
int_rtol = 1.0e-6
int_atol = 1.0e-4
scipy_int_method = 'RK45'
julia_int_method = 'Tsit5'
non_dimensionalize = False

In [ ]:
from TidalPy.tides.potential import tidal_potential_nsr, tidal_potential_nsr_modes, tidal_potential_simple

def tidal_potential_helper(potential_model, radius_matrix, longitude_matrix, colatitude_matrix, time_matrix,
                           orbital_frequency, spin_frequency, semi_major_axis, eccentricity):

    if potential_model == 'simple':
        modes = {'n': orbital_frequency}
        potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
                tidal_potential_simple(radius_matrix, longitude_matrix, colatitude_matrix, orbital_frequency, eccentricity, time_matrix)
    elif potential_model == 'nsr':
        modes = {'n': orbital_frequency}
        potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi = \
            tidal_potential_nsr(radius_matrix, longitude_matrix, colatitude_matrix, orbital_frequency, eccentricity, time_matrix,
                spin_frequency, world_radius=np.max(radius_matrix), host_mass=host_mass, semi_major_axis=semi_major_axis,
                use_static=False
                )
    elif potential_model == 'modes':
        modes, potential_dict, potential_dtheta_dict, potential_dphi_dict, potential_d2theta_dict,\
            potential_d2phi_dict, potential_dtheta_dphi_dict = \
            tidal_potential_nsr_modes(
                radius_matrix, longitude_matrix, colatitude_matrix, orbital_frequency, eccentricity, time_matrix,
                spin_frequency, world_radius=np.max(radius_matrix), host_mass=host_mass, semi_major_axis=semi_major_axis,
                use_static=False
                )
    else:
        raise ValueError('Unknown Model')

    return modes, potential, potential_dtheta, potential_dphi, potential_d2theta, potential_d2phi, potential_dtheta_dphi

## Homogeneous Planet


In [ ]:
from TidalPy.toolbox.multilayer.homogen_solid import calculate_homogen_solid

# Switches
use_static = False

# Planet Structure
radii = np.linspace(0.1, radius, planet_N)
densities = density_bulk * np.ones_like(radii)
volumes = (4. / 3.) * np.pi * (radius[1:]**3 - radius[:-1]**3)
masses = volumes * densities
mass_below = np.asarray([np.sum(masses[0:i]) for i in range(planet_N)])
gravities = G * mass_below / radii**2

# Find volumes for each voxel
dr = radii[1] - radii[0]
dColat_rad = colatitude_rad[1] - colatitude_rad[0]
dLong_rad = longitude_rad[1] - longitude_rad[0]
r_mtx, long_mtx, colat_mtx = np.meshgrid(radii, longitude_rad, colatitude_rad)
voxel_volumes = r_mtx**2 * np.sin(colat_mtx) * dr * dColat_rad * dLong_rad

breakpoint()


def homogen_planet(shear=mantle_shear, viscosity=mantle_viscosity, bulk_modulus=mantle_bulk_mod, orbital_frequency=orbital_freq_baseline,
                   rheology_func=mantle_rheology):

    # Convert floats to arrays if not already
    if type(shear) is float:
        shear = shear * np.ones_like(radii)
    if type(viscosity) is float:
        viscosity = viscosity * np.ones_like(radii)
    if type(bulk_modulus) is float:
        bulk_modulus = bulk_modulus * np.ones_like(radii)

    # Calculate other properties
    semi_major_axis = orbital_motion2semi_a(orbital_frequency, host_mass, mass)

    # Load rheology and calculate complex compliance
    complex_comp = known_complex_compliances[rheology_func](orbital_frequency, shear**-1, viscosity)
    complex_shear = complex_comp**(-1)

    # Calculate tidal ys
    tidal_y, tidal_y_derivative = \
        calculate_homogen_solid(
            radius=radii, shear_modulus=complex_shear, bulk_modulus=mantle_bulk_mod,
            density=densities, gravity=gravities, frequency=orbital_frequency,
            order_l=order_l, use_static=use_static,
            surface_boundary_condition=None,
            use_kamata=use_kamata, use_julia=use_julia,
            use_numba_integrator=use_numba_integrator,
            verbose=verbose,
            int_rtol=int_rtol, int_atol=int_atol, scipy_int_method=scipy_int_method,
            julia_int_method=julia_int_method,
            non_dimensionalize=non_dimensionalize,
            planet_bulk_density=density_bulk
        )

    # Build higher dimension domains
    time_domain = np.linspace(0., 86400. * rads2days(orbital_frequency), time_N)
    rad_mtx, time_mtx, long_mtx, colat_mtx = np.meshgrid(radii, time_domain, longitude_rad, colatitude_rad, indexing='ij')

    bulk_moduli_higher_dim, _, _, _ = \
        np.meshgrid(bulk_modulus, time_domain, longitude_rad, colatitude_rad, indexing='ij')
    radius_array_higher_dim, _, _, _ = \
        np.meshgrid(radii, time_domain, longitude_rad, colatitude_rad, indexing='ij')
    volume_array_higher_dim, _, _, _ = \
        np.meshgrid(volumes, time_domain, longitude_rad, colatitude_rad, indexing='ij')

    # For the voxel volumes we need to insert a new axis for time (since it already has ones for radius, lat, long).
    voxel_volume_array_higher_dim = np.repeat(voxel_volumes[:, np.newaxis, :, :], len(time_domain), axis=1)



## Surface Heating Maps

In [ ]:


tidal_y = convergence_ls_dynamic_liq(
    tidal_y_solutions_by_layer: TidalYSolType, surface_solution: np.ndarray,
    gravity_array_layer0: np.ndarray, density_array_layer0: np.ndarray,
    radius_array_layer0: np.ndarray, orbital_freq: float
    )



